# Blockhash Oracle Unified Deployment Script

This notebook deploys the blockhash oracle contracts on configured chains.
Supports incremental deployment - you can add new chains at any time.

## 1. Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

# Network type selection
NETWORK_TYPE = "testnets"  # "testnets" or "mainnets"

# Deployment mode
# - "full": Deploy all contracts
# - "auto": Deploy to all chains that don't have contracts yet
# - "manual": Only deploy to chains listed in CHAINS_TO_DEPLOY
DEPLOYMENT_MODE = "full"

# For manual mode, specify which chains to deploy to
# Example: ["base-sepolia", "optimism-sepolia"]
CHAINS_TO_DEPLOY = []

# Force redeployment of specific contracts on specific chains
# Format: {"contract_type": [list of chains to redeploy]}
# Example: {"BlockOracle": ["base-sepolia"], "LZBlockRelay": ["arbitrum-sepolia"]}
FORCE_REDEPLOY = {}

## 2. Initialize Environment

In [ ]:
import json
import os
import sys
import logging
from pathlib import Path

import boa
from boa.explorer import Etherscan
from dotenv import load_dotenv
from eth_account import Account
from web3 import Web3

# Add parent directory to path for imports
sys.path.append(str(Path().resolve().parent))
from ABIs import createX_abi

# Import from deployment folder
from LZMetadata import LZMetadata
from DeploymentManager import DeploymentManager

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Load environment variables
load_dotenv()

# Constants
CREATE_X_ADDRESS = "0xba5Ed099633D3B313e4D5F7bdc1305d3c28ba5Ed"
READ_CHANNEL_ID = 4294967295  # max uint32

## 3. Deployment State Management

In [ ]:
# Initialize deployment state manager
deployment_state = DeploymentManager()

## 4. Load Configuration

In [ ]:
# Load chains configuration
with open("chains.json", "r") as f:
    chains_config = json.load(f)

# Get chains for the selected network type
all_chains = chains_config[NETWORK_TYPE]

# Find main chain
main_chain = None
for chain_name, config in all_chains.items():
    if config.get("is_main_chain", False):
        main_chain = chain_name
        break

if not main_chain:
    raise ValueError(f"No main chain defined for {NETWORK_TYPE}")

logging.info(f"Main chain: {main_chain}")
logging.info(f"Total chains in config: {len(all_chains)}")

# Determine which chains to deploy to
deployed_chains = deployment_state.get_deployed_chains(NETWORK_TYPE)
logging.info(f"Already deployed chains: {deployed_chains}")

if DEPLOYMENT_MODE == "full":
    # Deploy to all chains that don't have deployments yet
    chains_to_deploy = set(all_chains.keys())

elif DEPLOYMENT_MODE == "auto":
    # Deploy to all chains that don't have deployments yet
    chains_to_deploy = set(all_chains.keys()) - deployed_chains
    # Always include chains that need redeployment
    for contract_type, chains in FORCE_REDEPLOY.items():
        chains_to_deploy.update(chains)

elif DEPLOYMENT_MODE == "manual":
    # Deploy only to specified chains
    chains_to_deploy = set(CHAINS_TO_DEPLOY)
    # Add chains that need redeployment
    for contract_type, chains in FORCE_REDEPLOY.items():
        chains_to_deploy.update(chains)
else:
    raise ValueError(f"Invalid DEPLOYMENT_MODE: {DEPLOYMENT_MODE}")

# Filter chains to only those we're deploying to
chains = {k: v for k, v in all_chains.items() if k in chains_to_deploy}

if not chains:
    logging.info("No chains to deploy to. All chains already have deployments.")
    print(
        "\nAll chains already have deployments. To redeploy specific contracts, use FORCE_REDEPLOY."
    )
else:
    logging.info(f"Chains to deploy to: {list(chains.keys())}")

## 5. Setup Deployer Account

In [ ]:
# Get deployer account
if NETWORK_TYPE == "testnets":
    private_key = os.environ.get("WEB3_TESTNET_PK")
    if not private_key:
        raise ValueError("WEB3_TESTNET_PK not found in environment")
    deployer = Account.from_key(private_key)
else:
    # For mainnets, use secure key utilities
    sys.path.append(os.path.expanduser("~/projects/keys/scripts"))
    from secure_key_utils import get_account_pk

    deployer = get_account_pk()

logging.info(f"Deployer address: {deployer.address}")

## 6. Initialize Chains State

In [ ]:
# Initialize LayerZero metadata
lz = LZMetadata()

# Initialize state dictionary
state_dict = {}

# Also load main chain if it's already deployed (needed for read config)
if main_chain in deployed_chains and main_chain not in chains:
    chains[main_chain] = all_chains[main_chain]

for chain_name, config in chains.items():
    state_dict[chain_name] = {}

    # Build RPC URL
    if "rpc" in config:
        rpc_url = config["rpc"]
    else:
        api_key = os.environ.get(config["rpc_env_var"])
        if not api_key:
            raise ValueError(f"{config['rpc_env_var']} not found in environment")
        rpc_url = config["rpc_template"].format(api_key)

    # Setup boa environment
    boa.set_network_env(rpc_url)
    boa.env.add_account(deployer)

    # Store state
    state_dict[chain_name]["config"] = config
    state_dict[chain_name]["rpc"] = rpc_url
    state_dict[chain_name]["boa"] = boa.env
    state_dict[chain_name]["w3"] = Web3(Web3.HTTPProvider(rpc_url))
    state_dict[chain_name]["createx"] = boa.loads_abi(createX_abi).at(CREATE_X_ADDRESS)
    state_dict[chain_name]["evm_version"] = config["evm_version"]

    # Get LayerZero metadata
    try:
        lz_metadata = lz.get_chain_metadata(chain_name)
        state_dict[chain_name]["eid"] = lz_metadata["metadata"]["eid"]
        state_dict[chain_name]["endpoint"] = lz_metadata["metadata"]["endpointV2"]
        state_dict[chain_name]["send_lib"] = lz_metadata["metadata"].get(
            "sendUln302", "unavailable"
        )
        state_dict[chain_name]["receive_lib"] = lz_metadata["metadata"].get(
            "receiveUln302", "unavailable"
        )
        state_dict[chain_name]["read_lib"] = lz_metadata["metadata"].get(
            "readLib1002", "unavailable"
        )
        state_dict[chain_name]["dvns"] = lz_metadata["dvns"]
        state_dict[chain_name]["executor"] = lz_metadata["metadata"].get(
            "executor", "0x0000000000000000000000000000000000000000"
        )
    except Exception as e:
        logging.warning(f"Failed to get LZ metadata for {chain_name}: {e}")

# Verify setup
for chain_name in state_dict.keys():
    with boa.swap_env(state_dict[chain_name]["boa"]):
        chain_id = boa.env.evm.patch.chain_id
        balance = state_dict[chain_name]["w3"].eth.get_balance(deployer.address) / 1e18
        logging.info(f"{chain_name}: Chain ID {chain_id}, Deployer balance {balance:.6f} ETH")

## 7. Deployment Helper Functions

In [ ]:
def deploy_or_load_contract(chain_name, contract_name, contract_path, salt, constructor_args=None):
    """
    Deploy a contract using CREATE3 or load it if already deployed at the computed address.

    Args:
        chain_name: Name of the chain
        contract_name: Name of the contract for logging/saving
        contract_path: Path to the contract file
        salt: Salt for CREATE3 deployment
        constructor_args: Optional constructor arguments (as bytes)

    Returns:
        tuple: (contract_instance, address, was_deployed)
    """
    full_path = os.path.abspath(contract_path)

    with boa.swap_env(state_dict[chain_name]["boa"]):
        # Load contract deployer
        contract_deployer = boa.load_partial(
            full_path, compiler_args={"evm_version": state_dict[chain_name]["evm_version"]}
        )

        # The salt we generate already has the deployer address as first 20 bytes and 00 as 21st byte
        # This matches SenderBytes.MsgSender with RedeployProtectionFlag.False
        # The CREATE3 _guard function will use _efficientHash: keccak256(bytes32(uint256(uint160(msg.sender))) || salt)

        # Create bytes32 from msg.sender address (left-padded with zeros)
        sender_bytes32 = Web3.to_bytes(hexstr="0x" + "0" * 24 + deployer.address[2:])
        # sender_bytes32 = Web3.to_bytes(hexstr="0x" + deployer.address[2:].lower() + "0" * 24)
        # Convert address to integer first (like uint160), then to bytes32
        # Compute guarded salt as the CREATE3 contract does with _efficientHash
        guarded_salt = Web3.keccak(sender_bytes32 + salt)

        # Compute CREATE3 address using the guarded salt
        computed_address = state_dict[chain_name]["createx"].computeCreate3Address(
            guarded_salt, CREATE_X_ADDRESS
        )

        # Get bytecode at the computed address
        deployed_code = state_dict[chain_name]["w3"].eth.get_code(computed_address)

        if len(deployed_code) > 0:
            # Contract already deployed at this address
            logging.info(f"{contract_name} already deployed at {computed_address} on {chain_name}")

            # Create contract instance
            contract = contract_deployer.at(computed_address)

            # Save to deployment state if not already saved
            existing_address = deployment_state.get_deployed_contract(
                NETWORK_TYPE, chain_name, contract_name
            )
            if not existing_address or existing_address.lower() != computed_address.lower():
                deployment_state.save_deployment(
                    NETWORK_TYPE, chain_name, contract_name, computed_address
                )
                logging.info(f"Added {contract_name} at {computed_address} to deployment state")

            return contract, computed_address, False
        else:
            # Deploy new contract
            logging.info(f"Deploying {contract_name} to {computed_address} on {chain_name}...")

            # Prepare deployment code
            deploycode = contract_deployer.compiler_data.bytecode
            if constructor_args:
                deploycode = deploycode + constructor_args

            # Deploy via CREATE3 using the original salt (not the guarded one)
            address = state_dict[chain_name]["createx"].deployCreate3(salt, deploycode)

            # Verify addresses match
            if address.lower() != computed_address.lower():
                raise ValueError(
                    f"Deployed address {address} doesn't match computed address {computed_address}"
                )

            # Create contract instance
            contract = contract_deployer.at(address)

            # Save deployment
            deployment_state.save_deployment(NETWORK_TYPE, chain_name, contract_name, address)
            logging.info(f"{contract_name} deployed at {address} on {chain_name}")

            return contract, address, True

## 8. Setup Etherscan Verifiers

In [ ]:
# Get Etherscan API key (unified v2 key)
ETHERSCAN_API_KEY = os.environ.get("ETHERSCAN_API_KEY")
if not ETHERSCAN_API_KEY:
    logging.warning("ETHERSCAN_API_KEY not found - contract verification will be skipped")

# Setup verifiers for each chain
for chain_name, state in state_dict.items():
    if ETHERSCAN_API_KEY and "explorer" in state["config"]:
        explorer_url = state["config"]["explorer"]
        chain_id = state["config"]["chain_id"]
        verifier = Etherscan(explorer_url + "?chainid=" + str(chain_id), ETHERSCAN_API_KEY)
        state["verifier"] = verifier
        logging.info(f"Verifier configured for {chain_name}")
    else:
        state["verifier"] = None

## 9. Generate or Load Deployment Salts

In [ ]:
# Generate or load salts for deterministic deployment
guard_bytes = bytes.fromhex(deployer.address[2:] + "00")

# Salt types we need
salt_types = ["view", "oracle", "header_verifier", "relay"]
salts = {}

for salt_type in salt_types:
    # Try to load existing salt
    saved_salt = deployment_state.get_salt(salt_type)
    if saved_salt:
        salts[salt_type] = bytes.fromhex(saved_salt)
        logging.info(f"Loaded existing salt for {salt_type}")
    else:
        # Generate new salt
        salt = guard_bytes + os.urandom(11)
        salts[salt_type] = salt
        deployment_state.save_salt(salt_type, salt)
        logging.info(f"Generated new salt for {salt_type}")

logging.info("Deployment salts ready")

## 10. Deploy Main Chain View Contract

In [ ]:
# Deploy or load MainnetBlockView on the main chain
view_contract_path = os.path.abspath("../../contracts/MainnetBlockView.vy")

if main_chain in chains_to_deploy:
    existing_view = deployment_state.get_deployed_contract(
        NETWORK_TYPE, main_chain, "MainnetBlockView"
    )
    should_deploy_view = not existing_view or (
        "MainnetBlockView" in FORCE_REDEPLOY and main_chain in FORCE_REDEPLOY["MainnetBlockView"]
    )

    if should_deploy_view:
        # Deploy or load contract using CREATE3
        contract, address, was_deployed = deploy_or_load_contract(
            main_chain, "MainnetBlockView", view_contract_path, salts["view"]
        )
        state_dict[main_chain]["oracle"] = contract
    else:
        logging.info(f"MainnetBlockView already deployed on {main_chain}")
        # Load existing contract
        with boa.swap_env(state_dict[main_chain]["boa"]):
            contract_deployer = boa.load_partial(
                view_contract_path,
                compiler_args={"evm_version": state_dict[main_chain]["evm_version"]},
            )
            state_dict[main_chain]["oracle"] = contract_deployer.at(existing_view)
else:
    # Load existing deployment if needed for read config
    existing_view = deployment_state.get_deployed_contract(
        NETWORK_TYPE, main_chain, "MainnetBlockView"
    )
    if existing_view and main_chain in state_dict:
        with boa.swap_env(state_dict[main_chain]["boa"]):
            contract_deployer = boa.load_partial(
                view_contract_path,
                compiler_args={"evm_version": state_dict[main_chain]["evm_version"]},
            )
            state_dict[main_chain]["oracle"] = contract_deployer.at(existing_view)
            logging.info(f"Loaded existing MainnetBlockView at {existing_view} on {main_chain}")

## 11. Deploy Oracle and HeaderVerifier on Other Chains

In [ ]:
# Deploy BlockOracle and HeaderVerifier on all non-main chains
block_oracle_path = os.path.abspath("../../contracts/BlockOracle.vy")
header_verifier_path = os.path.abspath("../../contracts/HeaderVerifier.vy")

for chain_name in chains_to_deploy:
    if chain_name == main_chain:
        continue

    # Deploy BlockOracle
    existing_oracle = deployment_state.get_deployed_contract(
        NETWORK_TYPE, chain_name, "BlockOracle"
    )
    should_deploy_oracle = not existing_oracle or (
        "BlockOracle" in FORCE_REDEPLOY and chain_name in FORCE_REDEPLOY["BlockOracle"]
    )

    if should_deploy_oracle:
        # Deploy or load contract using CREATE3
        oracle_contract, oracle_address, was_deployed = deploy_or_load_contract(
            chain_name, "BlockOracle", block_oracle_path, salts["oracle"]
        )
        state_dict[chain_name]["oracle"] = oracle_contract
    else:
        logging.info(f"BlockOracle already deployed at {existing_oracle} on {chain_name}")
        with boa.swap_env(state_dict[chain_name]["boa"]):
            oracle_deployer = boa.load_partial(
                block_oracle_path,
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["oracle"] = oracle_deployer.at(existing_oracle)

    # Deploy HeaderVerifier
    existing_verifier = deployment_state.get_deployed_contract(
        NETWORK_TYPE, chain_name, "HeaderVerifier"
    )
    should_deploy_verifier = not existing_verifier or (
        "HeaderVerifier" in FORCE_REDEPLOY and chain_name in FORCE_REDEPLOY["HeaderVerifier"]
    )

    if should_deploy_verifier:
        # Deploy or load contract using CREATE3
        verifier_contract, verifier_address, was_deployed = deploy_or_load_contract(
            chain_name, "HeaderVerifier", header_verifier_path, salts["header_verifier"]
        )
        state_dict[chain_name]["header_verifier"] = verifier_contract
    else:
        logging.info(f"HeaderVerifier already deployed at {existing_verifier} on {chain_name}")
        with boa.swap_env(state_dict[chain_name]["boa"]):
            verifier_deployer = boa.load_partial(
                header_verifier_path,
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["header_verifier"] = verifier_deployer.at(existing_verifier)

## 12. Deploy LZBlockRelay Contracts

In [ ]:
# Deploy LZBlockRelay on all non-main chains
relay_contract_path = os.path.abspath("../../contracts/messengers/LZBlockRelay.vy")

for chain_name in chains_to_deploy:
    if chain_name == main_chain:
        continue

    # Check if we need to deploy
    existing_relay = deployment_state.get_deployed_contract(
        NETWORK_TYPE, chain_name, "LZBlockRelay"
    )
    should_deploy_relay = not existing_relay or (
        "LZBlockRelay" in FORCE_REDEPLOY and chain_name in FORCE_REDEPLOY["LZBlockRelay"]
    )

    if should_deploy_relay:
        # Constructor arguments for LZBlockRelay
        constructor_args = boa.util.abi.abi_encode(
            "(address)", (state_dict[chain_name]["endpoint"],)
        )

        # Deploy or load contract using CREATE3
        relay_contract, relay_address, was_deployed = deploy_or_load_contract(
            chain_name, "LZBlockRelay", relay_contract_path, salts["relay"], constructor_args
        )
        state_dict[chain_name]["block_relay"] = relay_contract
    else:
        logging.info(f"LZBlockRelay already deployed at {existing_relay} on {chain_name}")
        with boa.swap_env(state_dict[chain_name]["boa"]):
            relay_deployer = boa.load_partial(
                relay_contract_path,
                compiler_args={"evm_version": state_dict[chain_name]["evm_version"]},
            )
            state_dict[chain_name]["block_relay"] = relay_deployer.at(existing_relay)

## 13. Deployment Summary

In [ ]:
# Create deployment summary
deployment_summary = deployment_state.get_deployment_summary(NETWORK_TYPE)
deployment_summary["deployer"] = deployer.address

# Print summary
print("\n" + "=" * 80)
print(f"DEPLOYMENT SUMMARY - {NETWORK_TYPE.upper()}")
print("=" * 80)
print(f"Timestamp: {deployment_summary['timestamp']}")
print(f"Deployer: {deployment_summary['deployer']}")
print(f"\nDeployed to {len(chains_to_deploy)} chain(s) in this session")
print(f"Total deployed chains: {deployment_summary['total_chains']}")
print("\nAll Deployed Contracts:")

for chain_name, contracts in deployment_summary["contracts"].items():
    print(f"\n{chain_name}:")
    for contract_name, address in contracts.items():
        print(f"  {contract_name}: {address}")

print("\n" + "=" * 80)
print("\nNext step: Run setup.ipynb to configure the deployed contracts")

## 14. Verify Contracts on Block Explorers

In [ ]:
# Verify all deployed contracts on block explorers
verification_results = {}

if ETHERSCAN_API_KEY:
    logging.info("\n" + "=" * 80)
    logging.info("Verifying contracts on block explorers...")
    logging.info("=" * 80)

    for chain_name in state_dict.keys():
        if not state_dict[chain_name].get("verifier"):
            logging.info(f"No verifier configured for {chain_name}, skipping")
            continue

        verification_results[chain_name] = {}

        with boa.swap_env(state_dict[chain_name]["boa"]):
            # Set verifier
            boa.set_verifier(state_dict[chain_name]["verifier"])

            # Check which contracts exist in state_dict for this chain
            if chain_name == main_chain:
                # Main chain only has oracle (MainnetBlockView)
                if "oracle" in state_dict[chain_name]:
                    try:
                        contract = state_dict[chain_name]["oracle"]
                        contract.ctor_calldata = b""
                        boa.verify(contract, verifier=state_dict[chain_name]["verifier"])
                        verification_results[chain_name]["MainnetBlockView"] = "Submitted"
                        logging.info(
                            f"Verification submitted for MainnetBlockView at {contract.address} on {chain_name}"
                        )
                    except Exception as e:
                        verification_results[chain_name]["MainnetBlockView"] = f"Failed: {str(e)}"
                        logging.error(f"Failed to verify MainnetBlockView on {chain_name}: {e}")
            else:
                # Non-main chains have oracle, header_verifier, and block_relay
                contracts_to_verify = [
                    ("oracle", "BlockOracle", b""),
                    ("header_verifier", "HeaderVerifier", b""),
                    ("block_relay", "LZBlockRelay", None),  # Constructor args handled separately
                ]

                for key, name, ctor_data in contracts_to_verify:
                    if key in state_dict[chain_name]:
                        try:
                            contract = state_dict[chain_name][key]

                            # Handle constructor args
                            if name == "LZBlockRelay":
                                # LZBlockRelay has constructor args
                                args = boa.util.abi.abi_encode(
                                    "(address)", (state_dict[chain_name]["endpoint"],)
                                )
                                contract.ctor_calldata = args
                            else:
                                contract.ctor_calldata = ctor_data

                            # Verify the contract
                            boa.verify(contract, verifier=state_dict[chain_name]["verifier"])
                            verification_results[chain_name][name] = "Submitted"
                            logging.info(
                                f"Verification submitted for {name} at {contract.address} on {chain_name}"
                            )

                        except Exception as e:
                            verification_results[chain_name][name] = f"Failed: {str(e)}"
                            logging.error(f"Failed to verify {name} on {chain_name}: {e}")

    # Print verification summary
    print("\n" + "=" * 80)
    print("VERIFICATION SUMMARY")
    print("=" * 80)

    for chain_name, results in verification_results.items():
        if results:  # Only show chains with verification attempts
            print(f"\n{chain_name}:")
            for contract_name, status in results.items():
                print(f"  {contract_name}: {status}")

    print("\n" + "=" * 80)
    print("\nNote: Verification may take a few minutes to complete on block explorers.")
    print(
        "You can check the status by visiting the contract addresses on the respective explorers."
    )
else:
    logging.warning("\nContract verification skipped - ETHERSCAN_API_KEY not found in environment")